# GBM(Gradient Boosting MAchine)

- 부스팅 알고리즘은 여러 개의 약한 학습기(weak learner)를 순차적으로 학습-예측하면서 잘못 예측한 데이터에 가중치 부여를 통해 오류를 개선해 나가면서 학습하는 방식이다. 부스팅의 대표적인 구현은 AdaBoost와 GradientBoost가 있다.
    - AdaBoost 는 오류 데이터에 가중치를 부여하면서 부스팅을 수행하는 대표적인 알고리즘이다.
        - 첫번째 학습기에 가중치 0.3, 두번째 학습기에 가중치 0.5, 세번째 학습기에 가중치 0.8을 부여한 후 모두 결합해 예측을 수행한다.
    - GradientBoost도 에이다 부스트와 유사하지만, 가중치 업데이트를 경사 하강법을 이용하는 것이 큰 차이이다.
        - 오류값은 실제값 - 예측값이다.
- GBM은 CART기반의 다른 알고리즘과 마찬가지로 분류와 회귀 모두 가능.
- 사이킷런은 GradientBoostingClassifier 클래스를 제공한다.

## 사이킷런의 GBM을 이용해 사용자 행동 데이터 세트를 예측 분류하기

In [14]:
from sklearn.ensemble import GradientBoostingClassifier
import time
import warnings
import pandas as pd
warnings.filterwarnings('ignore')

from sklearn.metrics import accuracy_score

In [11]:
# 중복된 피처 이름 바꾸기
def get_new_feature_name_df(old_feature_name_df):
    feature_dup_df = pd.DataFrame(data=old_feature_name_df.groupby('column_name').cumcount(),
                                 columns=['dup_cnt'])
    feature_dup_df = feature_dup_df.reset_index()
    
    new_feature_name_df = pd.merge(old_feature_name_df.reset_index(), feature_dup_df, how="outer")
    new_feature_name_df['column_name'] = new_feature_name_df[['column_name',
                                                              'dup_cnt']].apply(lambda x : x[0] + '_' + str(x[1]) 
                                                                                if x[1] > 0 else x[0], axis=1)
    new_feature_name_df = new_feature_name_df.drop(['index'], axis=1)
    return new_feature_name_df

In [12]:
# 데이터셋을 구성하는 함수 설정
def get_human_dataset():
    
    # 각 데이터 파일들은 공백으로 분리되어 있으므로 read_csv에서 공백문자를 sep으로 할당
    feature_name_df = pd.read_csv('./human_activity/features.txt', sep='\s+', 
                                  header=None, names=['column_index', 'column_name'])
    # 데이터프레임에 피처명을 컬럼으로 뷰여하기 위해 리스트 객체로 다시 반환
    new_feature_name_df = get_new_feature_name_df(feature_name_df)
    
    # 데이터프레임에 피처명을 칼럼으로 부여하기 위해 리스트로 다시 변환
    feature_name = new_feature_name_df.iloc[:,1].values.tolist()
    
    # 학습 피처 데이터셋과 테스트 피처 데이터를 DataFrame으로 로딩. 칼럼명은 feature_name
    X_train = pd.read_csv('./human_activity/train/X_train.txt', sep='\s+', 
                          header=None, names=feature_name)
    X_test = pd.read_csv('./human_activity/test/X_test.txt', sep='\s+', 
                          header=None, names=feature_name)
    
    y_train = pd.read_csv('./human_activity/train/y_train.txt', sep='\s+', 
                          header=None, names=['action'])
    y_test = pd.read_csv('./human_activity/test/y_test.txt', sep='\s+', 
                          header=None, names=['action'])
    #로드된 학습/테스트용 데이터 DataFrame 반환
    return X_train, X_test, y_train, y_test

In [15]:
# get_human_dataset()함수로 데이터 셋 가져오기
X_train, X_test, y_train, y_test = get_human_dataset()

# GBM수행시간 측정을 위함. 시작 시간 설정.
start_time = time.time()

gb_clf = GradientBoostingClassifier(random_state=0)
gb_clf.fit(X_train,y_train)
gb_pred = gb_clf.predict(X_test)
gb_accuracy = accuracy_score(y_test,gb_pred)

print('GBM 정확도: {0:.4f}'.format(gb_accuracy))
print('GBM 수행 시간: {0:.1f}초'.format(time.time() - start_time))

GBM 정확도: 0.9389
GBM 수행 시간: 1471.4초


- 수행시간 오래걸림.
- GBM이 랜덤 포레스트보다는 예측 성능이 조금 뛰어난 경우가 많다. 하지만 수행시간이 오래걸리고, 하이퍼 파라미터 튜닝 노력도 더 필요하다. (수행시간 문제는 GBM이 극복해야할 중요한 과제이다.)

## GBM 하이퍼 파라미터
- max_depth: 트리의 깊이 (int, default=None)
- max_features: 선택할 feature의 개수, 보통 default값으로 씀 (default='auto')
- loss : 경사 하강법에서 사용할 비용 함수를 지정. 특별한 이유가 없으면 기본값이 'deviance'그대로 적용
- learning_rate : GBM이 학습을 진행할 때마다 적용하는 학습률. weak learner가 순차적으로 오류값을 보정해 나가는데 적용하는 계수. 0~1사이의 값을 지정할 수 있으며, 기본값은 0.1이다.
    - 너무 작은 값을 적용하면 업데이트되는 값이 작아져서 최소 오류값을 찾아 예측 성능이 높아질 가능성이 높다. 하지만 많은 weak learner는 순차적인 반복이 필요해서 수행시간이 오래걸리고, 또 너무 작게 설정하면 모든 weak learner의 반복이 완료돼도 최소 오류값을 찾이 못할 수 있다. 반대로 큰 값을 적용하면 최소 오류 값을 찾지 못하고 그냥 지나쳐 버려 예측 성능이 떨어질 가능성이 높지만 빠른 수행은 가능.
    - 그래서 n_estimators와 상호보완적으로 조합해 사용. learning_rate를 작게하고 n_estimators를 크게하면 더이상 성능이 좋아지지 않는 한계점까지 예측 성능이 조금씩 좋아질 수 있다. 하지만 너무 오래걸리고 예측 성능이 현격히 좋아지진 않음.
- n_estimators: Weak learner의 개수. default=100
- subsample : weak learner가 학습에 사용하는 데이터 샘플링 비율. 기본값은 1(전체 학습데이터를 기반으로 학습한다는 의미)
    - 과적합이 염려되는 경우 subsample을 1보다 작게 설정